# Подвиг 6

In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Целевая функция, которую мы хотим аппроксимировать
def func(x):
    return 0.5 * x**2 - 0.1 * 1/np.exp(-x) + 0.5 * np.cos(2*x) - 2.

# Аппроксимация полиномом 3-й степени: w0 + w1*x + w2*x^2 + w3*x^3
def func_a(w, x):
    return w[0] + w[1]*x + w[2]*x**2 + w[3]*x**3

# ------------------------------------------------------
# 1) Подготовка данных
coord_x = np.arange(-4.0, 6.0, 0.1)   # сетка x от -4 до 6 с шагом 0.1
coord_y = func(coord_x)               # значения целевой функции на сетке
sz = len(coord_x)                     # число точек в выборке

# ------------------------------------------------------
# 2) Гиперпараметры обучения
eta = np.array([0.1, 0.01, 0.001, 0.0001])  # шаги обучения для каждого веса
w = np.array([0., 0., 0., 0.])              # начальные веса = 0
N = 500                                     # число итераций SGD
lm = 0.02                                   # λ для экспоненциального сглаживания ошибки
batch_size = 20                             # размер мини-батча
gamma = 0.8                                 # коэффициент импульса (momentum)
v = np.zeros(len(w))                        # начальный импульс (нулевой)
Qe = 0                                      # начальное значение скользящего среднего ошибки
np.random.seed(0)                           # фиксируем случайность для воспроизводимости

# ------------------------------------------------------
# 3) Итеративное обучение
for i in range(N):

    # ---- выбираем случайный мини-батч ----
    k = np.random.randint(0, sz - batch_size - 1)  # случайная стартовая позиция
    K = batch_size                                 # просто сокращение для удобства
    # создаём матрицу признаков (каждая строка = [1, x, x^2, x^3])
    x_k = np.column_stack([
        np.ones(K), 
        coord_x[k:k+K], 
        coord_x[k:k+K] ** 2, 
        coord_x[k:k+K] ** 3
    ])
    # целевые значения на батче
    y_k = coord_y[k:k+K]

    # ---- предсказание с учётом импульса Нестерова (lookahead) ----
    prediction = x_k @ (w - gamma * v)
    # x_k (K×4) @ (вектор весов (4,)) → получаем вектор предсказаний длины K

    # ---- вычисляем градиент ошибки ----
    grad = 2 / K * ((prediction - y_k) @ x_k)
    # формула: (2/K) * Σ (ŷ_i - y_i) * x_i
    # результат: вектор длины 4 (градиенты для каждого параметра w)

    # ---- обновляем импульс и веса ----
    v = gamma * v + (1 - gamma) * eta * grad   # обновляем накопленный импульс
    w = w - v                                  # сдвигаем веса в направлении -v

    # ---- обновляем сглаженное значение ошибки ----
    Qe = lm * np.mean((prediction - y_k) ** 2) + (1 - lm) * Qe
    # Qe = λ * (ошибка на батче) + (1-λ) * старое значение

# ------------------------------------------------------
# 4) После обучения
w = tuple(w)  # преобразуем веса в кортеж (если так требуется в задании)

# оцениваем среднюю квадратичную ошибку на всей выборке
Q = np.mean((coord_y - func_a(w, coord_x)) ** 2)

print(f"Найденные веса:, {w[0]:.3f}, {w[1]:.3f}, {w[2]:.3f}.")
print("Среднеквадратичная ошибка на всей выборке:", Q)


Найденные веса:, -2.125, 0.629, 0.388.
Среднеквадратичная ошибка на всей выборке: 4.022898737697549
